In [1]:
import matplotlib.pyplot as plt

import pandas as pd

from tqdm.auto import tqdm

from sklearn.preprocessing import LabelEncoder

## Load data

In [2]:
PA_data = pd.read_csv("../data/geoplant-at-paiss/GLC25_PA_metadata_train.csv")

PA_data["lon"] = PA_data["lon"] * (1 - PA_data["lat"] / 90) # As we closer to poles, the longitude distance is smaller
PA_data["lon"] = PA_data["lon"] * 111700 # Convert degrees to meters
PA_data["lat"] = PA_data["lat"] * 111700 # Convert degrees to meters

encoder = LabelEncoder()
PA_data["region"] = encoder.fit_transform(PA_data["region"])

In [3]:
PA_data = pd.read_csv("../data/geoplant-at-paiss/GLC25_PA_metadata_train.csv")[["speciesId", "region"]]
# P0_data = pd.read_csv("../data/geoplant-at-paiss/GLC25_P0_metadata_train.csv")[["speciesId", "region"]]

# all_data = pd.concat([P0_data, PA_data]).reset_index(drop=True)

# Group by survey_id (place speciesId in list)
aggregations = {"speciesId": lambda x: x.value_counts().keys().to_list()}

PA_data_grouped = PA_data.groupby("region").agg(aggregations).reset_index()

top_mapper = PA_data_grouped.set_index("region").to_dict()["speciesId"]
top_mapper_global = PA_data["speciesId"].value_counts().keys().to_list()

In [4]:
Test_data = pd.read_csv("../data/geoplant-at-paiss/GLC25_PA_metadata_test.csv")
Test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lon                14784 non-null  float64
 1   lat                14784 non-null  float64
 2   year               14784 non-null  int64  
 3   geoUncertaintyInM  13944 non-null  float64
 4   areaInM2           14140 non-null  float64
 5   region             14784 non-null  object 
 6   country            14784 non-null  object 
 7   surveyId           14784 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 924.1+ KB


In [5]:
predictions = []
for inputs in tqdm(Test_data["region"]):
    preds = top_mapper_global if inputs not in top_mapper else top_mapper[inputs]
    preds = preds[:52]
    predictions.append(preds)


submission = pd.DataFrame({
    "surveyId": Test_data.surveyId.values,
    "predictions": [
        " ".join(str(int(i)) for i in pred)
        for pred in predictions
    ]
})
submission.to_csv("submission.csv", index=False)

  0%|          | 0/14784 [00:00<?, ?it/s]